In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
with open('content3.html','r', encoding='utf-8') as f:
    response = f.read()
soup = BeautifulSoup(response, 'html.parser')


In [13]:
html_fragment = soup.find_all(class_='row_card')

In [29]:
html_fragment[1].find( class_='right').get_text(strip=True).replace('\n','').replace(' ','')

'02-2055/2016∼М-55672/2016'

In [16]:
html_fragment[1]

<div class="row_card">
<div class="left">                Номер дела ~ материала
    </div>
<div class="right">
                    02-2055/2016

            
                            ∼                                     М-55672/2016                                    
    </div>
</div>

In [131]:
dict_values = {
    'Уникальный идентификатор дела':None,
    'Номер дела ~ материала': None,
    'Номер жалобы': None,
    'Истец':None,
    "Ответчик": None,
    'Заявитель':None,
    'Cудья': None,
    'Статья КоАП РФ': None,
    'Суд, вынесший решение':None,
    'Категория дела': None,
    'Текущее состояние':None,
    'Результат рассмотрения':None

}

In [132]:
html_fragment = soup.find_all(class_='row_card')

for html in html_fragment:

    key = html.find(class_='left').get_text(strip=True)
    
    if key == 'Номер дела ~ материала':
        key = key.replace('\n','').replace(' ','')
        
    if key in dict_values.keys():
        value = html.find(class_='right').get_text(strip=True)
        dict_values[key] = value
        
    if key == 'Стороны':
        strong_tags = html.find_all('strong')
        for strong_tag in strong_tags:
            text = strong_tag.get_text(strip=True)
            next_sibling = strong_tag.next_sibling
            if 'Истец' in text[:5]:
                dict_values['Истец'] = next_sibling.strip() if next_sibling else None
            if 'Ответ' in text[:5]:
                dict_values['Ответчик'] = next_sibling.strip() if next_sibling else None
            if 'Заявител' in text[:8]:
                dict_values['Заявитель'] = next_sibling.strip() if next_sibling else None


dict_values

{'Уникальный идентификатор дела': '77RS0020-01-2016-008801-05',
 'Номер дела ~ материала': None,
 'Номер жалобы': None,
 'Истец': None,
 'Ответчик': None,
 'Заявитель': '"Сбербанк России" в лице филиала Московского банка ОАО "Сбербанк России"',
 'Cудья': 'Примак В.Г.',
 'Категория дела': '237 - Иные дела особого производства',
 'Текущее состояние': 'Удовлетворено, 30.03.2016',
 'Результат рассмотрения': None,
 'Суд, вынесший решение': None}

In [40]:
def get_info(html_fragment):
    dict_values = {
    'id': None,
    'Уникальный идентификатор дела':None,
    'Номер дела ~ материала': None,
    'Номер жалобы': None,
    'Истец':None,
    "Ответчик": None,
    'Заявитель':None,
    'Cудья': None,
    'Статья КоАП РФ': None,
    'Суд, вынесший решение':None,
    'Категория дела': None,
    'Текущее состояние':None,
    'Результат рассмотрения':None,
    'path':None}

    html_fragment = soup.find_all(class_='row_card') # ищем всю нужную информацию 

    for html in html_fragment:

        key = html.find(class_='left').get_text(strip=True)   
            
        if key in dict_values.keys():
            value = html.find(class_='right').get_text(strip=True)
            dict_values[key] = value
        
        if key == 'Номер дела ~ материала' or key == 'Номер дела':
            value_ = html.find(class_='right').get_text(strip=True).replace('\n','').replace(' ','')
            dict_values['Номер дела ~ материала'] = value_
            
        if key == 'Стороны': # обработка поля Стороны
            strong_tags = html.find_all('strong')
            for strong_tag in strong_tags:
                text = strong_tag.get_text(strip=True)
                next_sibling = strong_tag.next_sibling
                if 'Истец' in text[:5]:
                    dict_values['Истец'] = next_sibling.strip() if next_sibling else None
                if 'Ответ' in text[:5]:
                    dict_values['Ответчик'] = next_sibling.strip() if next_sibling else None
                if 'Заявител' in text[:8]:
                    dict_values['Заявитель'] = next_sibling.strip() if next_sibling else None

    return dict_values
get_info(soup)

{'id': None,
 'Уникальный идентификатор дела': '77RS0017-01-2016-016888-56',
 'Номер дела ~ материала': '02-2055/2016∼М-55672/2016',
 'Номер жалобы': None,
 'Истец': 'Сбербанк России',
 'Ответчик': 'Касаримова А. Н.',
 'Заявитель': None,
 'Cудья': 'Клинцова И.В.',
 'Статья КоАП РФ': None,
 'Суд, вынесший решение': None,
 'Категория дела': '213 - О взыскании сумм по договору займа, кредитному договору',
 'Текущее состояние': 'Удовлетворено, 02.03.2016',
 'Результат рассмотрения': None,
 'path': None}

In [12]:
info = get_info(soup)
[list(info.values())]

[['77RS0017-01-2016-016888-56',
  None,
  None,
  'Сбербанк России',
  'Касаримова А. Н.',
  None,
  'Клинцова И.В.',
  None,
  None,
  '213 - О взыскании сумм по договору займа, кредитному договору',
  'Удовлетворено, 02.03.2016',
  None]]

In [53]:
suggested_filename = 'abd'
save_directory = rf'C:\Data\Visual Studio\Kursova_2\documents\{suggested_filename}'
path_for_bd = save_directory.split('\\')[-2:]
path_for_bd = '\\'.join(path_for_bd)
path_for_bd

SyntaxError: EOL while scanning string literal (1812394387.py, line 4)

In [ ]:
# Уникальный_идентификатор_дела varchar(50) DEFAULT NULL,
#             Номер_дела varchar(50) DEFAULT NULL,
#             Истец varchar(200) DEFAULT NULL,
#             Ответчик varchar(200) DEFAULT NULL,
#             Судья varchar(20) DEFAULT NULL,
#             Статья_КоАП_РФ varchar(50) DEFAULT NULL,
#             Категория_дела varchar(100) DEFAULT NULL,
#             Текущее_состояние varchar(100) DEFAULT NULL,
#             Результат_рассмотрения varchar(100) DEFAULT NULL,
#             Ссылка_на_документ varchar(50) DEFAULT NULL
#             Номер жалобы
#               Суд, вынесший решение

In [93]:
from bs4 import BeautifulSoup

html_fragment = '''
<div class="right"><strong>Истец: </strong>Сбербанк России<br/><strong>Ответчик: </strong>Касаримова А. Н.<br/></div>
'''

soup = BeautifulSoup(html_fragment, 'html.parser')

# Функция для извлечения информации об истце и ответчике
def extract_party_info(html_content):
    # Парсинг HTML-фрагмента
    soup = BeautifulSoup(html_content, 'html.parser')
    right_div = soup.find('div', class_='right')
    if not right_div:
        return "Информация не найдена."
    
    result = {}
    
    # Ищем все теги <strong> и анализируем их содержимое
    strong_tags = right_div.find_all('strong')
    for strong_tag in strong_tags:
        text = strong_tag.get_text(strip=True)
        next_sibling = strong_tag.next_sibling
        
        if 'Истец:' in text:
            result['Истец'] = next_sibling.strip() if next_sibling else 'не указан'
        elif 'Ответчик:' in text:
            result['Ответчик'] = next_sibling.strip() if next_sibling else 'не указан'
    
    return result

info = extract_party_info(html_fragment)

for key, value in info.items():
    print(f"{key}: {value}")


Истец: Сбербанк России
Ответчик: Касаримова А. Н.


In [95]:
soup = BeautifulSoup(html_fragment, 'html.parser')
right_div = soup.find('div', class_='right')
if not right_div:
    print("Информация не найдена.")

result = {}

# Ищем все теги <strong> и анализируем их содержимое
strong_tags = right_div.find_all('strong')
for strong_tag in strong_tags:
    text = strong_tag.get_text(strip=True)
    next_sibling = strong_tag.next_sibling
    
    if 'Истец' in text[:5]:
        result['Истец'] = next_sibling.strip() if next_sibling else 'не указан'
    elif 'Ответ' in text[:5]:
        result['Ответчик'] = next_sibling.strip() if next_sibling else 'не указан'

In [99]:
strong_tags[0].get_text(strip=True)

'Истец:'

In [100]:
strong_tags[0].next_sibling

'Сбербанк России'

In [110]:
'Ответ:' in text

False

In [116]:
'Истец:'[:5]


'Истец'

In [113]:
"Ответчик"[:5]

'Ответ'

## Преобразование doc в docx

In [1]:
# pip install -U pypiwin32
import os
from win32com import client as wc

In [2]:
# Получение текущего рабочего каталога
current_dir = os.getcwd()

# Формирование пути к папке documents, которая находится в текущем рабочем каталоге
documents_dir = os.path.join(current_dir, 'documents')

# Изменение текущего рабочего каталога на documents
os.chdir(documents_dir)

# Проверка текущего рабочего каталога
print("Текущий рабочий каталог:", os.getcwd())

Текущий рабочий каталог: c:\Data\Visual Studio\Kursova_2\documents


In [60]:
w = wc.Dispatch('Word.Application')

 
paths_doc = []
paths_docs = []
paths_rtfx = []

folder = os.getcwd()

for root, dirs, files in os.walk(folder):
    for file in files:
        # if file.endswith('doc') :
        #     paths_doc.append(os.path.join(root, file))
        # if file.endswith('docx') :
        #     paths_docs.append(os.path.join(root, file))
        if file.endswith('rtfx') :
            paths_rtfx.append(os.path.join(root, file))

 

for path in paths_rtfx:
    doc = w.Documents.Open(path)
    doc.SaveAs(path+"x", 16)
    doc.Close()

w.Quit()


In [56]:
# for doc in paths_doc:
#     if doc + 'x' in paths_docs:
#         os.remove(doc)

# for i in paths_rtfx:
#     os.remove(i)

## Парсинг Документов

Поиск упоминаний законов с указанием номера и даты:

Паттерн: r"Закон РФ от \d{2}\.\d{2}\.\d{4} года №\d+-\d+ «[А-Яа-яёЁ\s]+»"
Поиск статей Гражданского кодекса РФ:

Паттерн для статьи: r"ст\. \d+ ГК РФ"
Паттерн для нескольких статей: r"ст\.ст\. \d+ и \d+ ГК РФ" или для диапазона: r"ст\.ст\. \d+–\d+ ГК РФ"
Поиск Федеральных законов с номером и датой:

Паттерн: r"Федеральный закон от \d{2}\.\d{2}\.\d{4} года №\d+-\d+ «[А-Яа-яёЁ\s]+»"
Поиск упоминаний Гражданского процессуального кодекса РФ:

Паттерн для статьи: r"ст\. \d+ ГПК РФ"
Паттерн для нескольких статей: r"ст\.ст\. \d+ и \d+ ГПК РФ" или для диапазона: r"ст\.ст\. \d+–\d+ ГПК РФ"

In [12]:

import docx
import re
import textract
# path = r'C:\Data\Visual Studio\Kursova_2\documents\Дело 33-412224_2020. Определение суда апелляционной инстанции. документ - обезличенная копия.doc'
path = r'documents\test1.doc'

In [43]:
path = r'documents\Дело 02-0316_2016. Решение. документ - обезличенная копия.rtf'

doc = docx.Document(path)

full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)

# Преобразуйте список строк в одну строку текста
full_text_string = '\n'.join(full_text)

# print(full_text_string)
# full_text_string
# full_text_string

PackageNotFoundError: Package not found at 'documents\Дело 02-0316_2016. Решение. документ - обезличенная копия.rtf'

In [42]:



# Текст документа, предполагается, что он уже загружен в full_text_string
# full_text_string = "Ваш текст здесь..."

# Список паттернов для поиска
patterns = [
    r"Закон РФ от \d{2}\.\d{2}\.\d{4} года №\d+-\d+ «[А-Яа-яёЁ\s]+»",
    r"ст\. \d+ ГК РФ",
    r"ст\.\d+\s*ГК\s*РФ",
    r"ст\.ст\. \d+ и \d+ ГК РФ",
    r"ст\.ст\. \d+–\d+ ГК РФ",
    r"\d+ ГК РФ",
    r"Федеральный закон от \d{2}\.\d{2}\.\d{4} года №\d+-\d+ «[А-Яа-яёЁ\s]+»",
    r"ст\. \d+ ГПК РФ",
    r"ст\.ст\. \d+ и \d+ ГПК РФ",
    r"ст\.ст\.\d+–\d+ ГПК РФ",
    r"ст\. ст\. \d+-\d+ ГПК РФ",
    r"ст\. ст\. \d+, \d+ ГПК РФ",
    r"ст\.ст\. \d+, \d+ ГПК РФ",
    r"\d+ Гражданского Кодекса РФ",
    r"\d+ ГПК РФ"
]

# Словарь для хранения результатов
found_laws = {"Законы": [], "Статьи ГК РФ": [], "Федеральные законы": [], "Статьи ГПК РФ": []}

# Поиск и добавление результатов
for pattern in patterns:
    matches = re.findall(pattern, full_text_string)
    if "Закон РФ" in pattern or "Федеральный закон" in pattern:
        found_laws["Законы"].extend(matches)
    elif "ГК" in pattern or 'Гражданского Кодекса РФ' in pattern:
        found_laws["Статьи ГК РФ"].extend(matches)
    elif "ГПК" in pattern:
        found_laws["Статьи ГПК РФ"].extend(matches)

# Вывод результатов
for key, value in found_laws.items():
    print(f"{key}:")
    for v in value:
        print(f" - {v}")
# found_laws


Законы:
Статьи ГК РФ:
 - ст. 15 ГК РФ
 - ст. 333 ГК РФ
 - 15 ГК РФ
 - 333 ГК РФ
Федеральные законы:
Статьи ГПК РФ:
 - ст. 39 ГПК РФ
 - 39 ГПК РФ
 - 199 ГПК РФ


In [72]:
re.findall(r"ст. \d+ Федерального закона", full_text_string)

['ст. 9 Федерального закона']

## Обновляем ссылки на документы

In [2]:
import sqlite3

# Подключение к базе данных
conn = sqlite3.connect('legal_cases.db')
cursor = conn.cursor()

# SQL запрос для обновления данных
update_query = """
UPDATE collection
SET Ссылка_на_документ = CASE
    WHEN substr(Ссылка_на_документ, -4) = '.doc'
    THEN Ссылка_на_документ || 'x'
    ELSE Ссылка_на_документ
END
WHERE Ссылка_на_документ LIKE '%.doc';
"""

try:
    # Выполнение запроса
    cursor.execute(update_query)
    # Сохранение изменений
    conn.commit()
    print("Обновление выполнено успешно.")
except Exception as e:
    # В случае ошибки выводим ее
    print("Ошибка при выполнении обновления:", e)

# Закрытие соединения с базой данных
conn.close()


Обновление выполнено успешно.


In [44]:
# Подключение к базе данных
conn = sqlite3.connect('legal_cases.db')
cursor = conn.cursor()

# SQL запрос для обновления данных
update_query = """
UPDATE collection
SET Ссылка_на_документ = CASE
    WHEN substr(Ссылка_на_документ, -4) = '.rtf'
    THEN Ссылка_на_документ || 'x'
    ELSE Ссылка_на_документ
END
WHERE Ссылка_на_документ LIKE '%.rtf';
"""

try:
    # Выполнение запроса
    cursor.execute(update_query)
    # Сохранение изменений
    conn.commit()
    print("Обновление выполнено успешно.")
except Exception as e:
    # В случае ошибки выводим ее
    print("Ошибка при выполнении обновления:", e)

# Закрытие соединения с базой данных
conn.close()


Обновление выполнено успешно.


In [50]:
import sqlite3

# Подключение к базе данных
conn = sqlite3.connect('legal_cases.db')
cursor = conn.cursor()

# SQL команда для добавления нового столбца
add_column_query = """
ALTER TABLE collection
ADD COLUMN Законы varchar(1000);
"""

try:
    # Выполнение команды
    cursor.execute(add_column_query)
    conn.commit()  # Сохраняем изменения в базе данных
    print("Столбец 'Законы' успешно добавлен.")
except sqlite3.OperationalError as e:
    print("Ошибка при добавлении столбца:", e)
finally:
    # Закрытие соединения с базой данных
    conn.close()


Столбец 'Законы' успешно добавлен.


In [7]:
# import sqlite3

# # Подключение к базе данных
# conn = sqlite3.connect('legal_cases.db')
# cursor = conn.cursor()

# # SQL запрос для удаления записей с ID от 1 до 5
# delete_query = """
# DELETE FROM collection WHERE id BETWEEN 1 AND 5;
# """

# try:
#     # Выполнение запроса
#     cursor.execute(delete_query)
#     # Сохранение изменений
#     conn.commit()
#     print(f"Удалено записей: {cursor.rowcount}")
# except Exception as e:
#     # В случае ошибки выводим ее
#     print("Ошибка при выполнении удаления:", e)

# # Закрытие соединения с базой данных
# conn.close()


Удалено записей: 5


## Парсим законы и вставляем в БД

In [45]:
import sqlite3

# Подключение к базе данных
conn = sqlite3.connect('legal_cases.db')
cursor = conn.cursor()

# SQL запрос для выборки данных
select_query = """
SELECT id, Ссылка_на_документ FROM collection;
"""


cursor.execute(select_query)
results = cursor.fetchall()


# Закрытие соединения с базой данных
conn.close()


In [46]:
results[:5]

[(6,
  'documents\\Дело 02-1918_2016. Определение о вступлении в дело третьего лица с са... документ - обезличенная копия.docx'),
 (7,
  'documents\\Дело 33-26985_2016. Определение суда апелляционной инстанции. документ - обезличенная копия.docx'),
 (8,
  'documents\\Дело 33-14953_2014. Определение суда апелляционной инстанции. документ - обезличенная копия.docx'),
 (9,
  'documents\\Дело 33-30635_2017. Определение суда апелляционной инстанции. документ - обезличенная копия.docx'),
 (10,
  'documents\\Дело 33-17847_2017. Определение суда апелляционной инстанции. документ - обезличенная копия.docx')]

In [18]:
def parser_doc(delo:list) ->dict:
    ID = delo[0]
    path = delo[1]
    try:
    
        doc = docx.Document(path)

        patterns = [
            r"Закон РФ от \d{2}\.\d{2}\.\d{4} года №\d+-\d+ «[А-Яа-яёЁ\s]+»",
            r"ст\. \d+ ГК РФ",
            r"ст\.\d+\s*ГК\s*РФ",
            r"ст\.ст\. \d+ и \d+ ГК РФ",
            r"ст\.ст\. \d+–\d+ ГК РФ",
            r"\d+ ГК РФ",
            r"Федеральный закон от \d{2}\.\d{2}\.\d{4} года №\d+-\d+ «[А-Яа-яёЁ\s]+»",
            r"ст\. \d+ ГПК РФ",
            r"ст\.ст\. \d+ и \d+ ГПК РФ",
            r"ст\.ст\.\d+–\d+ ГПК РФ",
            r"ст\. ст\. \d+-\d+ ГПК РФ",
            r"ст\. ст\. \d+, \d+ ГПК РФ",
            r"ст\.ст\. \d+, \d+ ГПК РФ",
            r"\d+ Гражданского Кодекса РФ",
            r"\d+ ГПК РФ"
        ]

        full_text = []
        for para in doc.paragraphs:
            full_text.append(para.text)

        # Преобразуйте список строк в одну строку текста
        full_text_string = '\n'.join(full_text)

        # Словарь для хранения результатов
        found_laws = {"Законы": [], "Статьи ГК РФ": [], "Федеральные законы": [], "Статьи ГПК РФ": []}

        # Поиск и добавление результатов
        for pattern in patterns:
            matches = re.findall(pattern, full_text_string)
            if "Закон РФ" in pattern or "Федеральный закон" in pattern:
                found_laws["Законы"].extend(matches)
            elif "ГК" in pattern or 'Гражданского Кодекса РФ' in pattern:
                found_laws["Статьи ГК РФ"].extend(matches)
            elif "ГПК" in pattern:
                found_laws["Статьи ГПК РФ"].extend(matches)
        return (ID, found_laws)
    except:
         return (ID, {})

In [19]:
parser_doc(results[2])

(8,
 {'Законы': [],
  'Статьи ГК РФ': ['ст.845 ГК РФ', 'ст.847 ГК РФ', '845 ГК РФ', '847 ГК РФ'],
  'Федеральные законы': [],
  'Статьи ГПК РФ': ['39 ГПК РФ', '330 ГПК РФ']})

In [47]:
%%time
# results = results[:10]
parsing = []
for doc in results:
    parsing.append(parser_doc(doc))
parsing

CPU times: total: 22.2 s
Wall time: 29.7 s


[(6,
  {'Законы': [],
   'Статьи ГК РФ': ['ст. 363 ГК РФ', '363 ГК РФ'],
   'Федеральные законы': [],
   'Статьи ГПК РФ': ['ст. 67 ГПК РФ',
    'ст. 423 ГПК РФ',
    'ст. 426 ГПК РФ',
    '67 ГПК РФ',
    '423 ГПК РФ',
    '426 ГПК РФ',
    '427 ГПК РФ']}),
 (7,
  {'Законы': [],
   'Статьи ГК РФ': ['ст. 848 ГК РФ',
    'ст. 850 ГК РФ',
    'ст. 820 ГК РФ',
    'ст. 432 ГК РФ',
    'ст. 434 ГК РФ',
    'ст. 435 ГК РФ',
    'ст. 421 ГК РФ',
    'ст. 430 ГК РФ',
    'ст. 437 ГК РФ',
    'ст. 438 ГК РФ',
    'ст. 432 ГК РФ',
    'ст. 395 ГК РФ',
    '848 ГК РФ',
    '849 ГК РФ',
    '850 ГК РФ',
    '820 ГК РФ',
    '432 ГК РФ',
    '434 ГК РФ',
    '438 ГК РФ',
    '435 ГК РФ',
    '438 ГК РФ',
    '421 ГК РФ',
    '430 ГК РФ',
    '437 ГК РФ',
    '438 ГК РФ',
    '432 ГК РФ',
    '395 ГК РФ'],
   'Федеральные законы': [],
   'Статьи ГПК РФ': ['ст. 103 ГПК РФ',
    'ст.ст. 329, 330 ГПК РФ',
    '103 ГПК РФ',
    '330 ГПК РФ']}),
 (8,
  {'Законы': [],
   'Статьи ГК РФ': ['ст.845 ГК РФ', '

In [48]:
parsing[-1]

(1947,
 {'Законы': [],
  'Статьи ГК РФ': ['ст.333 ГК РФ',
   'ст.820 ГК РФ',
   'ст.309 ГК РФ',
   'ст.330 ГК РФ',
   'ст.819 ГК РФ',
   'ст.333 ГК РФ',
   'ст.450 ГК РФ',
   '333 ГК РФ',
   '820 ГК РФ',
   '309 ГК РФ',
   '330 ГК РФ',
   '819 ГК РФ',
   '333 ГК РФ',
   '450 ГК РФ',
   '333 ГК РФ',
   '820 ГК РФ'],
  'Федеральные законы': [],
  'Статьи ГПК РФ': ['ст. 199 ГПК РФ',
   '199 ГПК РФ',
   '167 ГПК РФ',
   '57 ГПК РФ',
   '150 ГПК РФ',
   '98 ГПК РФ',
   '103 ГПК РФ',
   '198 ГПК РФ']})

In [51]:
import sqlite3
import json

def insert_or_update_json_data(db_name, table_name, data):
    # Подключение к базе данных
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Подготовка запроса на вставку или обновление
    query = f"""
    INSERT INTO {table_name} (id, Законы)
    VALUES (?, ?)
    ON CONFLICT(id) DO UPDATE SET
    Законы = excluded.Законы;
    """
    
    # Преобразуем словарь в JSON и вставляем данные
    values_to_insert = [(item[0], json.dumps(item[1], ensure_ascii=False)) for item in data]
    
    try:
        cursor.executemany(query, values_to_insert)
        conn.commit()
        print("Данные успешно вставлены/обновлены.")
    except Exception as e:
        print("Ошибка при вставке данных:", e)
    finally:
        conn.close()

# Пример использования
db_name = 'legal_cases.db'
table_name = 'collection'
insert_or_update_json_data(db_name, table_name, parsing)


Данные успешно вставлены/обновлены.
